In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import numpy as np
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from torch import optim, nn

from plotting import plot_model_outputs

import beer

output_notebook()

In [ ]:
mean = np.array([3., 3.])
cov = np.array([[2., 1.5], [1.5, 2.]])
data = np.random.multivariate_normal(mean, cov, size=100)

fig = figure(
    title='Linear subspace',
    width=400,
    height=400,
)
fig.circle(data[:, 0], data[:, 1])
show(fig)

In [ ]:
obs_dim = 2
latent_dim = 2
nb_samples = 10

enc_struct = nn.Sequential(
    nn.Linear(obs_dim, 2),
    nn.Tanh(),
    nn.Linear(2, 2),
    nn.Tanh()
)
encoder = beer.models.GaussianMLP(enc_struct, 2, latent_dim)

dec_struct = nn.Sequential(
    nn.Linear(latent_dim, 2),
    nn.Tanh(),
    nn.Linear(2, 2),
    nn.Tanh()
)
decoder = beer.models.GaussianMLP(dec_struct, 2, obs_dim)

#latent_model = beer.models.NormalDiagonalCovariance.create(
#    latent_dim,
#    mean_prec=1e-3,
#    prec_shape=1.,
#    prec_rate=1
#)
latent_model = beer.models.NaturalIsotropicGaussian(2)

model = beer.models.VAE(encoder, decoder, latent_model, nb_samples)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
history = beer.inference.History(report_interval=250)

In [ ]:
beer.inference.run_training(data, model, optimizer, 5000, history, batch_size=20, lrate_latent_model=0.)

In [ ]:
plot_model_outputs(model, data)

In [ ]:
history.plot()

In [ ]:
print(model.latent_model.exp_mean)
print(1/model.latent_model.exp_precision)
print(model.latent_model.exp_log_precision)

In [ ]:
np.log(181)